In [1]:
import sys,os
add_path = os.path.abspath('../python')
if not add_path in sys.path:
    sys.path.append(add_path)
print(sys.path.index(add_path))

14


In [2]:
import time
import numpy as np
from numpy import vstack, hstack
from segbench import csaAssign, correspondPixels,correspondVoxels, matchEdgeMaps2D, matchEdgeMaps3D

In [3]:
data = np.array([[1, 4, 3],
    [2, 5, 3],
    [3, 6, 3],
    [1, 5, 1],
    [2, 6, 1],
    [3, 4, 5]])
n = 6
e1 = csaAssign(n,data)
print e1

[[ 1.  5.  1.]
 [ 2.  6.  1.]
 [ 3.  4.  5.]]


In [4]:
# Unit test for csaAssign.
n = 1000
dg = np.random.rand(n,n)>.5
m = np.sum(dg.flat)
i = np.array(dg.ravel().nonzero()).astype(float)
g1 = 1+np.floor(np.divide(i,n))
g2 = (1+np.floor(np.divide(i,n)),
     1+ np.mod(i,n)+n,
     1+np.floor(np.random.rand(1,m))*1000)
g2 = np.array(np.vstack(g2)).T
t = time.time()
e2 = csaAssign(n*2,g2)
print '{:.3e}s'.format(time.time() - t)
esum = np.sum(e2,axis=0)
if not all(esum[0:2]==(n*(n+1)/2,n*(n+1)/2+n*n)): raise ValueError("bug")
if not np.sum(esum[0:2])==(2*n*(2*n+1)/2): raise ValueError("bug")
print n,m, e2[3][:]

6.949e-02s
1000 500130 [  4.00000000e+00   1.99300000e+03   1.00000000e+00]


In [5]:
# Unit test for correspondPixels.
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,oc,match1,match2 = correspondPixels(bmap1,bmap2,.005)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print npix,ncor,nout
        if ncor+nout!=npix: raise ValueError("bug")

        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 1.32e-03s 2 0 2
n=1 thresh=1.11e-01 5.70e-05s 2 0 2
n=1 thresh=2.22e-01 3.10e-05s 1 0 1
n=1 thresh=3.33e-01 5.20e-05s 2 0 2
n=1 thresh=4.44e-01 3.00e-05s 1 0 1
n=1 thresh=5.56e-01 3.10e-05s 1 0 1
n=1 thresh=6.67e-01 2.91e-05s 0 0 0
n=1 thresh=7.78e-01 2.81e-05s 1 0 1
n=1 thresh=8.89e-01 3.50e-05s 0 0 0
n=1 thresh=1.00e+00 4.89e-05s 0 0 0
n=2 thresh=0.00e+00 9.80e-05s 8 6 2
n=2 thresh=1.11e-01 1.81e-04s 8 6 2
n=2 thresh=2.22e-01 1.53e-04s 8 6 2
n=2 thresh=3.33e-01 5.98e-05s 5 2 3
n=2 thresh=4.44e-01 5.91e-05s 5 2 3
n=2 thresh=5.56e-01 4.91e-05s 4 2 2
n=2 thresh=6.67e-01 3.10e-05s 2 0 2
n=2 thresh=7.78e-01 3.10e-05s 3 0 3
n=2 thresh=8.89e-01 5.58e-05s 0 0 0
n=2 thresh=1.00e+00 3.29e-05s 0 0 0
n=3 thresh=0.00e+00 2.11e-04s 18 16 2
n=3 thresh=1.11e-01 2.20e-04s 16 12 4
n=3 thresh=2.22e-01 2.09e-04s 13 8 5
n=3 thresh=3.33e-01 2.47e-04s 14 12 2
n=3 thresh=4.44e-01 9.82e-05s 7 2 5
n=3 thresh=5.56e-01 8.80e-05s 7 2 5
n=3 thresh=6.67e-01 1.01e-04s 5 2 3
n=3 thresh=7.78e-01 8

In [6]:
# Unit test for matchEdgeMaps2D.
for n in np.round(np.logspace(0,2,10)).astype(int):
    for thresh in np.linspace(0,1,10):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n)>thresh).astype(float)
        t = time.time()
        cost,match1,match2 = matchEdgeMaps2D(bmap1,bmap2,.005,100)
        print '{:1.2e}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=0.00e+00 1.02e-04s 0 + 2 = 2 == 2
n=1 thresh=1.11e-01 5.60e-05s 0 + 2 = 2 == 2
n=1 thresh=2.22e-01 2.88e-05s 0 + 1 = 1 == 1
n=1 thresh=3.33e-01 5.29e-05s 0 + 2 = 2 == 2
n=1 thresh=4.44e-01 3.70e-05s 0 + 1 = 1 == 1
n=1 thresh=5.56e-01 5.70e-05s 0 + 2 = 2 == 2
n=1 thresh=6.67e-01 2.50e-05s 0 + 0 = 0 == 0
n=1 thresh=7.78e-01 2.91e-05s 0 + 0 = 0 == 0
n=1 thresh=8.89e-01 2.79e-05s 0 + 0 = 0 == 0
n=1 thresh=1.00e+00 3.10e-05s 0 + 0 = 0 == 0
n=2 thresh=0.00e+00 9.20e-05s 6 + 2 = 8 == 8
n=2 thresh=1.11e-01 6.60e-05s 6 + 1 = 7 == 7
n=2 thresh=2.22e-01 7.01e-05s 2 + 4 = 6 == 6
n=2 thresh=3.33e-01 6.01e-05s 4 + 1 = 5 == 5
n=2 thresh=4.44e-01 5.39e-05s 2 + 2 = 4 == 4
n=2 thresh=5.56e-01 3.31e-05s 0 + 3 = 3 == 3
n=2 thresh=6.67e-01 2.98e-05s 0 + 2 = 2 == 2
n=2 thresh=7.78e-01 5.48e-05s 4 + 1 = 5 == 5
n=2 thresh=8.89e-01 3.10e-05s 0 + 2 = 2 == 2
n=2 thresh=1.00e+00 2.60e-05s 0 + 0 = 0 == 0
n=3 thresh=0.00e+00 2.22e-04s 16 + 2 = 18 == 18
n=3 thresh=1.11e-01 1.72e-04s 12 + 4 = 16 == 16
n=3 

In [7]:
# Unit test for correspondVoxels
for n in np.round(np.logspace(0,6,4,base=2)).astype(int):
    for thresh in np.linspace(.5,.9,4):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n,n)>thresh).astype(float)

        t = time.time()
        cost,oc,match1,match2 = correspondVoxels(bmap1,bmap2,.005)
        print '{:1.4f}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=5.00e-01 0.0001s 2 + 0 = 2 == 2
n=1 thresh=6.33e-01 0.0000s 0 + 1 = 1 == 1
n=1 thresh=7.67e-01 0.0000s 0 + 0 = 0 == 0
n=1 thresh=9.00e-01 0.0000s 0 + 0 = 0 == 0
n=4 thresh=5.00e-01 0.0003s 34 + 28 = 62 == 62
n=4 thresh=6.33e-01 0.0001s 12 + 38 = 50 == 50
n=4 thresh=7.67e-01 0.0001s 6 + 23 = 29 == 29
n=4 thresh=9.00e-01 0.0000s 0 + 7 = 7 == 7
n=16 thresh=5.00e-01 0.0406s 2018 + 2061 = 4079 == 4079
n=16 thresh=6.33e-01 0.0236s 1190 + 1884 = 3074 == 3074
n=16 thresh=7.67e-01 0.0070s 434 + 1445 = 1879 == 1879
n=16 thresh=9.00e-01 0.0015s 98 + 707 = 805 == 805
n=64 thresh=5.00e-01 9.7403s 130884 + 131329 = 262213 == 262213
n=64 thresh=6.33e-01 3.4553s 70100 + 121800 = 191900 == 191900
n=64 thresh=7.67e-01 1.2439s 28474 + 93913 = 122387 == 122387
n=64 thresh=9.00e-01 0.1635s 5222 + 47354 = 52576 == 52576


In [8]:
# Unit test for matchEdgeMaps3D
for n in np.round(np.logspace(0,6,4,base=2)).astype(int):
    for thresh in np.linspace(.5,.9,4):
        print 'n={:d} thresh={:1.2e}'.format(n,thresh),
        bmap1 = (np.random.rand(n,n,n)>thresh).astype(float)
        bmap2 = (np.random.rand(n,n,n)>thresh).astype(float)

        t = time.time()
        cost,match1,match2 = matchEdgeMaps3D(bmap1,bmap2,.005*np.sqrt(n**3),100)
        print '{:1.4f}s'.format(time.time() - t),
        npix = len(bmap1.nonzero()[0]) + len(bmap2.nonzero()[0])
        ncor = len(match1.nonzero()[0]) + len(match1.nonzero()[0])
        nout = len(((bmap1>0)&(match1==0)).nonzero()[0]) + len(((bmap2>0)&(match2==0)).nonzero()[0])
        print ncor,'+',nout,'=',ncor+nout,'==',npix
        if ncor+nout!=npix: raise ValueError("bug")
        
        match1f = match1.ravel()
        idx1 = match1f.astype(int)
        a1 = match1f.nonzero()[0]
        a2 = match1f[idx1.nonzero()[0]].ravel()
        a = vstack((a1,a2)).T

        match2f = match2.ravel()
        idx2 = match2f.astype(int)
        b1 = match2f.nonzero()[0]
        b2 = match2f[idx2.nonzero()[0]].ravel()
        b = vstack((b2,b1)).T
        b = b[b[:,0].argsort(),]
        if np.sum(a-b): raise ValueError("bug")

n=1 thresh=5.00e-01 0.0043s 2 + 0 = 2 == 2
n=1 thresh=6.33e-01 0.0000s 0 + 0 = 0 == 0
n=1 thresh=7.67e-01 0.0000s 0 + 0 = 0 == 0
n=1 thresh=9.00e-01 0.0000s 0 + 0 = 0 == 0
n=4 thresh=5.00e-01 0.0003s 28 + 37 = 65 == 65
n=4 thresh=6.33e-01 0.0002s 14 + 31 = 45 == 45
n=4 thresh=7.67e-01 0.0001s 6 + 20 = 26 == 26
n=4 thresh=9.00e-01 0.0001s 8 + 15 = 23 == 23
n=16 thresh=5.00e-01 0.0417s 2046 + 2050 = 4096 == 4096
n=16 thresh=6.33e-01 0.0231s 1130 + 1877 = 3007 == 3007
n=16 thresh=7.67e-01 0.0066s 454 + 1484 = 1938 == 1938
n=16 thresh=9.00e-01 0.0015s 82 + 730 = 812 == 812
n=64 thresh=5.00e-01 31.7409s 130928 + 131151 = 262079 == 262079
n=64 thresh=6.33e-01 25.7225s 70730 + 121897 = 192627 == 192627
n=64 thresh=7.67e-01 23.0356s 28522 + 93828 = 122350 == 122350
n=64 thresh=9.00e-01 9.9525s 5338 + 47387 = 52725 == 52725


n=10 thresh=5.00e-01 6.77e-04s 98 48 50
